In [1]:
%load_ext autoreload
%autoreload 2

In [40]:
from collections import Counter
import glob
import os
import pickle
import re
import string
import time

import pandas as pd
import numpy as np

from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from gensim.models.ldamulticore import LdaMulticore
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from sklearn.manifold import TSNE

import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 16})

NOTEBOOKS_DIR = os.path.abspath(os.getcwd())
ROOT_DIR = os.path.split(NOTEBOOKS_DIR)[0]
PROCESSED_DATA_DIR = os.path.join(ROOT_DIR, 'data', 'processed')

FINAL_DF_FILEPATH = os.path.join(PROCESSED_DATA_DIR, 'final.csv')
ML_ONLY_FILEPATH = os.path.join(PROCESSED_DATA_DIR, 'machine_learning_only.csv')

In [7]:
df_ml = pd.read_csv(ML_ONLY_FILEPATH, encoding='utf-8')
df_ml = df_ml.reset_index()

In [4]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_ml = tfidf_vectorizer.fit_transform(df_ml['description'])
features = np.array(tfidf_vectorizer.get_feature_names())
nmf_model = NMF(n_components=10, random_state=42)
W = nmf_model.fit_transform(tfidf_ml)
H = nmf_model.components_

In [8]:
document_idx = 0

# topic loadings for the document
print(W[document_idx])

# document title
print(df_ml['title'][document_idx])

# document description
print(df_ml['description'][document_idx])

[0.01750071 0.03036693 0.         0.         0.02932531 0.00416817
 0.         0.00764464 0.         0.00019623]
A Unified SVM Framework for Signal Estimation
  This paper presents a unified framework to tackle estimation problems in
Digital Signal Processing (DSP) using Support Vector Machines (SVMs). The use
of SVMs in estimation problems has been traditionally limited to its mere use
as a black-box model. Noting such limitations in the literature, we take
advantage of several properties of Mercer's kernels and functional analysis to
develop a family of SVM methods for estimation in DSP. Three types of signal
model equations are analyzed. First, when a specific time-signal structure is
assumed to model the underlying system that generated the data, the linear
signal model (so called Primal Signal Model formulation) is first stated and
analyzed. Then, non-linear versions of the signal structure can be readily
developed by following two different approaches. On the one hand, the signal

In [34]:
list(tfidf_vectorizer.transform([df_ml['description'][document_idx]]).toarray()[0])[:10]

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [18]:
H.shape

(10, 60744)

In [17]:
features

array(['00', '000', '00000', ..., 'zyody', 'zyx', 'zzstefan'],
      dtype='<U65')

### Create a dictionary where we can look up a word and get its loadings for each topic

In [19]:
word_topic_loading_lookup = {}
for i in range(len(features)):
    word_topic_loading_lookup[features[i]] = H[:, i]

In [21]:
word_topic_loading_lookup['data']

array([3.42728368, 0.        , 0.        , 0.        , 0.03507072,
       0.01918562, 0.04993902, 0.        , 0.29730119, 0.        ])

In [26]:
document_words = tfidf_vectorizer.inverse_transform(tfidf_vectorizer.transform([df_ml['description'][document_idx]]))[0]
document_words[:10]

array(['written', 'versions', 'vector', 'using', 'usefulness', 'used',
       'use', 'unified', 'underlying', 'types'], dtype='<U65')

In [33]:
for word in document_words:
    print(f'{word:17}: {np.round(word_topic_loading_lookup[word], 3)}')

written          : [0.008 0.007 0.006 0.003 0.003 0.    0.    0.018 0.001 0.   ]
versions         : [0.01  0.027 0.006 0.003 0.006 0.    0.003 0.003 0.006 0.024]
vector           : [0.126 0.106 0.    0.    0.    0.058 0.    0.129 0.002 0.092]
using            : [0.423 0.13  0.203 0.119 0.199 0.011 0.046 0.264 0.056 0.08 ]
usefulness       : [0.034 0.008 0.    0.006 0.015 0.    0.    0.011 0.01  0.005]
used             : [0.398 0.089 0.133 0.073 0.141 0.016 0.044 0.158 0.083 0.048]
use              : [0.286 0.05  0.125 0.133 0.15  0.015 0.029 0.156 0.017 0.04 ]
unified          : [0.022 0.042 0.004 0.013 0.02  0.022 0.005 0.034 0.018 0.001]
underlying       : [0.115 0.051 0.002 0.018 0.096 0.096 0.009 0.    0.046 0.058]
types            : [0.133 0.    0.048 0.008 0.021 0.044 0.027 0.054 0.017 0.003]
traditionally    : [0.02  0.002 0.008 0.011 0.003 0.002 0.    0.011 0.001 0.   ]
time             : [0.768 0.    0.235 0.069 0.104 0.017 0.    0.    0.024 0.311]
tackle           : [0.04  0.

In [51]:
translation = str.maketrans(string.punctuation, ' '*len(string.punctuation))
paper_words = df_ml['description'][document_idx].lower().translate(translation).split()
for word in paper_words:
    word_loadings = word_topic_loading_lookup.get(word)
    if word_loadings is not None:
        print(f'{word:17}: {np.round(word_loadings, 2)}')
    else:
        print(word)

this
paper            : [0.34 0.21 0.14 0.13 0.08 0.06 0.06 0.21 0.11 0.12]
presents         : [0.1  0.01 0.02 0.03 0.01 0.   0.   0.04 0.02 0.01]
a
unified          : [0.02 0.04 0.   0.01 0.02 0.02 0.   0.03 0.02 0.  ]
framework        : [0.27 0.16 0.01 0.22 0.18 0.14 0.08 0.16 0.04 0.01]
to
tackle           : [0.04 0.02 0.   0.02 0.   0.01 0.   0.05 0.   0.  ]
estimation       : [0.09 0.17 0.   0.   0.3  0.   0.   0.   0.01 0.16]
problems         : [0.13 0.65 0.   0.18 0.   0.04 0.   0.04 0.03 0.12]
in
digital          : [0.03 0.   0.01 0.   0.   0.   0.   0.01 0.   0.  ]
signal           : [0.08 0.14 0.03 0.   0.   0.09 0.   0.08 0.   0.03]
processing       : [0.16 0.03 0.09 0.   0.   0.03 0.   0.12 0.   0.  ]
dsp              : [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
using            : [0.42 0.13 0.2  0.12 0.2  0.01 0.05 0.26 0.06 0.08]
support          : [0.2  0.08 0.   0.   0.   0.   0.   0.03 0.   0.07]
vector           : [0.13 0.11 0.   0.   0.   0.06 0.   0.13 0.   0.09]
machines     

### Color each topic loading by intensity

In [131]:
for i in range(30, 100):
    for j in range(0, 100):
        print(f"\033[0;{i};{j}m Color test", f" -- ({i}, {j}) \n")

 Color test  -- (30, 0) 

 Color test  -- (30, 1) 

 Color test  -- (30, 2) 

 Color test  -- (30, 3) 

 Color test  -- (30, 4) 

 Color test  -- (30, 5) 

 Color test  -- (30, 6) 

 Color test  -- (30, 7) 

 Color test  -- (30, 8) 

 Color test  -- (30, 9) 

 Color test  -- (30, 10) 

 Color test  -- (30, 11) 

 Color test  -- (30, 12) 

 Color test  -- (30, 13) 

 Color test  -- (30, 14) 

 Color test  -- (30, 15) 

 Color test  -- (30, 16) 

 Color test  -- (30, 17) 

 Color test  -- (30, 18) 

 Color test  -- (30, 19) 

 Color test  -- (30, 20) 

 Color test  -- (30, 21) 

 Color test  -- (30, 22) 

 Color test  -- (30, 23) 

 Color test  -- (30, 24) 

 Color test  -- (30, 25) 

 Color test  -- (30, 26) 

 Color test  -- (30, 27) 

 Color test  -- (30, 28) 

 Color test  -- (30, 29) 

 Color test  -- (30, 30) 

 Color test  -- (30, 31) 

 Color test  -- (30, 32) 

 Color test  -- (30, 33) 

 Color test  -- (30, 34) 

 Color test  -- (30, 35) 

 Color test  -- (30, 36) 

 Color test

 Color test  -- (44, 65) 

 Color test  -- (44, 66) 

 Color test  -- (44, 67) 

 Color test  -- (44, 68) 

 Color test  -- (44, 69) 

 Color test  -- (44, 70) 

 Color test  -- (44, 71) 

 Color test  -- (44, 72) 

 Color test  -- (44, 73) 

 Color test  -- (44, 74) 

 Color test  -- (44, 75) 

 Color test  -- (44, 76) 

 Color test  -- (44, 77) 

 Color test  -- (44, 78) 

 Color test  -- (44, 79) 

 Color test  -- (44, 80) 

 Color test  -- (44, 81) 

 Color test  -- (44, 82) 

 Color test  -- (44, 83) 

 Color test  -- (44, 84) 

 Color test  -- (44, 85) 

 Color test  -- (44, 86) 

 Color test  -- (44, 87) 

 Color test  -- (44, 88) 

 Color test  -- (44, 89) 

 Color test  -- (44, 90) 

 Color test  -- (44, 91) 

 Color test  -- (44, 92) 

 Color test  -- (44, 93) 

 Color test  -- (44, 94) 

 Color test  -- (44, 95) 

 Color test  -- (44, 96) 

 Color test  -- (44, 97) 

 Color test  -- (44, 98) 

 Color test  -- (44, 99) 

 Color test  -- (45, 0) 

 Color test  -- (45, 1) 

 Co


 Color test  -- (57, 71) 

 Color test  -- (57, 72) 

 Color test  -- (57, 73) 

 Color test  -- (57, 74) 

 Color test  -- (57, 75) 

 Color test  -- (57, 76) 

 Color test  -- (57, 77) 

 Color test  -- (57, 78) 

 Color test  -- (57, 79) 

 Color test  -- (57, 80) 

 Color test  -- (57, 81) 

 Color test  -- (57, 82) 

 Color test  -- (57, 83) 

 Color test  -- (57, 84) 

 Color test  -- (57, 85) 

 Color test  -- (57, 86) 

 Color test  -- (57, 87) 

 Color test  -- (57, 88) 

 Color test  -- (57, 89) 

 Color test  -- (57, 90) 

 Color test  -- (57, 91) 

 Color test  -- (57, 92) 

 Color test  -- (57, 93) 

 Color test  -- (57, 94) 

 Color test  -- (57, 95) 

 Color test  -- (57, 96) 

 Color test  -- (57, 97) 

 Color test  -- (57, 98) 

 Color test  -- (57, 99) 

 Color test  -- (58, 0) 

 Color test  -- (58, 1) 

 Color test  -- (58, 2) 

 Color test  -- (58, 3) 

 Color test  -- (58, 4) 

 Color test  -- (58, 5) 

 Color test  -- (58, 6) 

 Color test  -- (58, 7) 

 Color t


 Color test  -- (73, 14) 

 Color test  -- (73, 15) 

 Color test  -- (73, 16) 

 Color test  -- (73, 17) 

 Color test  -- (73, 18) 

 Color test  -- (73, 19) 

 Color test  -- (73, 20) 

 Color test  -- (73, 21) 

 Color test  -- (73, 22) 

 Color test  -- (73, 23) 

 Color test  -- (73, 24) 

 Color test  -- (73, 25) 

 Color test  -- (73, 26) 

 Color test  -- (73, 27) 

 Color test  -- (73, 28) 

 Color test  -- (73, 29) 

 Color test  -- (73, 30) 

 Color test  -- (73, 31) 

 Color test  -- (73, 32) 

 Color test  -- (73, 33) 

 Color test  -- (73, 34) 

 Color test  -- (73, 35) 

 Color test  -- (73, 36) 

 Color test  -- (73, 37) 

 Color test  -- (73, 38) 

 Color test  -- (73, 39) 

 Color test  -- (73, 40) 

 Color test  -- (73, 41) 

 Color test  -- (73, 42) 

 Color test  -- (73, 43) 

 Color test  -- (73, 44) 

 Color test  -- (73, 45) 

 Color test  -- (73, 46) 

 Color test  -- (73, 47) 

 Color test  -- (73, 48) 

 Color test  -- (73, 49) 

 Color test  -- (73, 50) 



 Color test  -- (86, 17) 

 Color test  -- (86, 18) 

 Color test  -- (86, 19) 

 Color test  -- (86, 20) 

 Color test  -- (86, 21) 

 Color test  -- (86, 22) 

 Color test  -- (86, 23) 

 Color test  -- (86, 24) 

 Color test  -- (86, 25) 

 Color test  -- (86, 26) 

 Color test  -- (86, 27) 

 Color test  -- (86, 28) 

 Color test  -- (86, 29) 

 Color test  -- (86, 30) 

 Color test  -- (86, 31) 

 Color test  -- (86, 32) 

 Color test  -- (86, 33) 

 Color test  -- (86, 34) 

 Color test  -- (86, 35) 

 Color test  -- (86, 36) 

 Color test  -- (86, 37) 

 Color test  -- (86, 38) 

 Color test  -- (86, 39) 

 Color test  -- (86, 40) 

 Color test  -- (86, 41) 

 Color test  -- (86, 42) 

 Color test  -- (86, 43) 

 Color test  -- (86, 44) 

 Color test  -- (86, 45) 

 Color test  -- (86, 46) 

 Color test  -- (86, 47) 

 Color test  -- (86, 48) 

 Color test  -- (86, 49) 

 Color test  -- (86, 50) 

 Color test  -- (86, 51) 

 Color test  -- (86, 52) 

 Color test  -- (86, 53) 

 

In [156]:
d = {
    0.1: "\033[0;30;47m {:.2f} \033[0;30;0m",
    0.5: "\033[0;30;46m {:.2f} \033[0;30;0m",
    1: "\033[0;30;43m {:.2f} \033[0;30;0m",
    2: "\033[0;30;45m {:.2f} \033[0;30;0m",
    3: "\033[0;30;41m {:.2f} \033[0;30;0m"
}

In [157]:
translation = str.maketrans(string.punctuation, ' '*len(string.punctuation))
paper_words = df_ml['description'][document_idx].lower().translate(translation).split()
for word in paper_words:
    num_row = []
    word_loadings = word_topic_loading_lookup.get(word)
    if word_loadings is not None:
        for num in word_loadings:
            if num < 0.1:
                num_row.append(d[0.1].format(round(num, 2)))
            elif num < 0.5:
                num_row.append(d[0.5].format(round(num, 2)))
            elif num < 1:
                num_row.append(d[1].format(round(num, 2)))
            elif num < 2:
                num_row.append(d[2].format(round(num, 2)))
            else:
                num_row.append(d[3].format(round(num, 2)))
        print(f'{word:17}:', *[num_row[i] for i in range(len(num_row))])
    else:
        print(word)

this
paper            :  0.34   0.21   0.14   0.13   0.08   0.06   0.06   0.21   0.11   0.12 
presents         :  0.10   0.01   0.02   0.03   0.01   0.00   0.00   0.04   0.02   0.01 
a
unified          :  0.02   0.04   0.00   0.01   0.02   0.02   0.00   0.03   0.02   0.00 
framework        :  0.27   0.16   0.01   0.22   0.18   0.14   0.08   0.16   0.04   0.01 
to
tackle           :  0.04   0.02   0.00   0.02   0.00   0.01   0.00   0.05   0.00   0.00 
estimation       :  0.09   0.17   0.00   0.00   0.30   0.00   0.00   0.00   0.01   0.16 
problems         :  0.13   0.65   0.00   0.18   0.00   0.04   0.00   0.04   0.03   0.12 
in
digital          :  0.03   0.00   0.01   0.00   0.00   0.00   0.00   0.01   0.00   0.00 
signal           :  0.08   0.14   0.03   0.00   0.00   0.09   0.00   0.08   0.00   0.03 
processing       :  0.16   0.03   0.09   0.00   0.00   0.03   0.00   0.12   0.00   0.00 
dsp              :  0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00 
using   

## Can we color the actual _word_ in the paragraph? Perhaps one topic at a time.

In [216]:
word_color_mappings = {
    0.1: "\033[0;30;47m{}\033[0;30;0m",
    0.5: "\033[0;30;46m{}\033[0;30;0m",
    1: "\033[0;30;43m{}\033[0;30;0m",
    2: "\033[0;30;45m{}\033[0;30;0m",
    3: "\033[0;30;41m{}\033[0;30;0m"
}

In [211]:
def get_colorful_words(topic_idx, paper_words, word_topic_loading_lookup, word_color_mappings):
    colorful_paper_description = []
    for word in paper_words:
        word_loadings = word_topic_loading_lookup.get(word)
        if word_loadings is not None:
            topic_loading = word_loadings[topic_idx]
            if topic_loading < 0.1:
                colorful_paper_description.append(word_color_mappings[0.1].format(word))
            elif topic_loading < 0.5:
                colorful_paper_description.append(word_color_mappings[0.5].format(word))
            elif topic_loading < 1:
                colorful_paper_description.append(word_color_mappings[1].format(word))
            elif topic_loading < 2:
                colorful_paper_description.append(word_color_mappings[2].format(word))
            else:
                colorful_paper_description.append(word_color_mappings[3].format(word))
        else:
            colorful_paper_description.append(word)
    
    return colorful_paper_description

def print_article_colored_by_word_loadings(document_idx, topic_labels=None):
    translation = str.maketrans(string.punctuation, ' '*len(string.punctuation))
    paper_words = df_ml['description'][document_idx].lower().translate(translation).split()
    for topic_idx in range(10):
        colorful_words = get_colorful_words(topic_idx, paper_words, word_topic_loading_lookup, word_color_mappings)
        if topic_labels:
            print(f'TOPIC {topic_idx} - {topic_labels[topic_idx]}')
        else:
            print(f'TOPIC {topic_idx}')
        print()
        print(' '.join([colorful_words[i] for i in range(len(colorful_words))]))
        print()
        print('*' * 70)
        print()

In [245]:
hand_labeled_features = [
    'machine learning / time series', #0
    'gradient / optimization / convergence', #1
    'neural networks / deep learning', #2
    'reinforcement learning', #3
    'variational bayesian', #4
    'graphs / graph ML', #5
    'ML attacks / GANs', #6
    'image / text / classification', #7
    'clustering', #8
    'algorithms / regret / optimization' #9
]

def softmax(v, temperature=1.0):
    '''
    A heuristic to convert arbitrary positive values into probabilities.
    See: https://en.wikipedia.org/wiki/Softmax_function
    '''
    expv = np.exp(v / temperature)
    s = np.sum(expv)
    return expv / s

def analyze_article(paper_idx, descriptions, titles, W, hand_labels):
    '''
    Print an analysis of a single NYT articles, including the article text
    and a summary of which topics it represents. The topics are identified
    via the hand-labels which were assigned by the user.
    '''
    print('Title:', titles[paper_idx])
    print()
    print('Description:\n', descriptions[paper_idx])
    probs = softmax(W[paper_idx], temperature=0.01)
    for prob, label in zip(probs, hand_labels):
        print('--> {:.2f}% {}'.format(prob * 100, label))
    print()

def get_document_report(document_idx, df_ml, topic_labels=None):
    analyze_article(document_idx, df_ml['description'], df_ml['title'], W, hand_labeled_features)
    
    print()
    print('*' * 70)
    print()

    # print paragraphs, one for each topic, words colored by topic loadings
    print_article_colored_by_word_loadings(document_idx, topic_labels=topic_labels)

In [246]:
# This is one of the research papers that is most directionally aligned with only topics 3 and 5
get_document_report(23856, df_ml, topic_labels=hand_labeled_features)

Title: Graph Convolutional Reinforcement Learning

Description:
   Learning to cooperate is crucially important in multi-agent environments. The
key is to understand the mutual interplay between agents. However, multi-agent
environments are highly dynamic, which makes it hard to learn abstract
representations of their mutual interplay. To tackle these difficulties, we
propose graph convolutional reinforcement learning, where graph convolution
adapts to the dynamics of the underlying graph of the multi-agent environment,
and relation kernels capture the interplay between agents by their relation
representations. Latent features produced by convolutional layers from
gradually increased receptive fields are exploited to learn cooperation, and
cooperation is further boosted by temporal relation regularization for
consistency. Empirically, we show that our method substantially outperforms
existing methods in a variety of cooperative scenarios.

--> 0.21% machine learning / time series
--> 0

In [184]:
# This is one of the research papers that is most directionally aligned with only topic 7
get_document_report(21108, df_ml)

[0.         0.         0.         0.         0.         0.
 0.         0.07293617 0.         0.        ]
Unsupervised Domain Adaptation of Contextualized Embeddings for Sequence
  Labeling
  Contextualized word embeddings such as ELMo and BERT provide a foundation for
strong performance across a wide range of natural language processing tasks by
pretraining on large corpora of unlabeled text. However, the applicability of
this approach is unknown when the target domain varies substantially from the
pretraining corpus. We are specifically interested in the scenario in which
labeled data is available in only a canonical source domain such as newstext,
and the target domain is distinct from both the labeled and pretraining texts.
To address this scenario, we propose domain-adaptive fine-tuning, in which the
contextualized embeddings are adapted by masked language modeling on text from
the target domain. We test this approach on sequence labeling in two
challenging domains: Early Modern En

In [189]:
# This is the research paper with the highest summed topic loadings
get_document_report(25878, df_ml)

[0.         0.         0.00882468 0.0104415  0.         0.1303849
 0.00109208 0.0049476  0.10071876 0.        ]
Attributed Graph Clustering: A Deep Attentional Embedding Approach
  Graph clustering is a fundamental task which discovers communities or groups
in networks. Recent studies have mostly focused on developing deep learning
approaches to learn a compact graph embedding, upon which classic clustering
methods like k-means or spectral clustering algorithms are applied. These
two-step frameworks are difficult to manipulate and usually lead to suboptimal
performance, mainly because the graph embedding is not goal-directed, i.e.,
designed for the specific clustering task. In this paper, we propose a
goal-directed deep learning approach, Deep Attentional Embedded Graph
Clustering (DAEGC for short). Our method focuses on attributed graphs to
sufficiently explore the two sides of information in graphs. By employing an
attention network to capture the importance of the neighboring nodes 